In [ ]:
import torch
import pandas as pd
import astropy as ap
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
# from astropy.io import fits
import pdb
from scipy.ndimage.filters import maximum_filter1d
import glob
import fitsio as fits
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.distributions.normal import Normal
from tqdm import tqdm

In [ ]:
import sys
sys.path.insert(0, '../')
sys.path.insert(1, '../latent_ode/')
import latent_ode.lib as ode
import latent_ode.lib.utils as utils
from latent_ode.lib.latent_ode import LatentODE
from latent_ode.lib.encoder_decoder import Encoder_z0_ODE_RNN, Decoder
from latent_ode.lib.diffeq_solver import DiffeqSolver
from latent_ode.lib.ode_func import ODEFunc

In [ ]:
from latent_rnn import create_LatentODE_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

## Loading

In [ ]:
obsrv_std = torch.Tensor([0.01]).to(device)
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))
input_dim = 1
model = create_LatentODE_model(input_dim, z0_prior, obsrv_std)

In [ ]:
state = torch.load('latent_ode_state.pth.tar')
# state = torch.load('latent_ode_state.pth.tar', map_location=torch.device('cpu'))

In [ ]:
model.load_state_dict(state['state_dict'])

In [ ]:
model.eval()

In [ ]:
loader = torch.load('gaia_train.pt')

## Inference

In [ ]:
z_us = []
z_stds = []
truths = []
ts = []
recs = []
for batch in tqdm(loader):
    observed = batch['observed_data']
    true = batch['data_to_predict']
    truths.append(true)
    mask = batch['observed_mask']
    mask_pred = batch['mask_predicted_data']
#     mask_pred = torch.ones(mask.shape)
    x = torch.cat((observed, mask), -1)
    x2 = torch.cat((true, mask_pred), -1)
    t = batch['observed_tp']
    ts.append(t.detach().cpu())
    z_u, z_std = model.encoder_z0.forward(x, t)
    rec = model.get_reconstruction(time_steps_to_predict=t, truth=x, truth_time_steps=t)
    recs.append(rec[0].detach().cpu())
#     z_u = z_u.squeeze()
    z_u = z_u.detach().cpu()
    z_std = z_std.detach().cpu()
    z_us.append(z_u)
    z_stds.append(z_std)

In [ ]:
z_us = torch.cat(z_us,1).squeeze()
z_stds = torch.cat(z_stds,1).squeeze()
truths = torch.cat(truths,0).squeeze()

In [ ]:
ts[0]

In [ ]:
truths.shape

In [ ]:
recs = torch.cat(recs,1).squeeze()

In [ ]:
recs.shape

In [ ]:
z_us.shape, z_stds.shape, truths.shape

In [ ]:
ix = 88

In [ ]:
u = z_us[ix]
std = z_stds[ix]

In [ ]:
def sample(z, t):
    sol_y = model.diffeq_solver.sample_traj_from_prior(z, t, n_traj_samples = 1)
    out = model.decoder(sol_y)
    return out

In [ ]:
recs = recs.squeeze()

In [ ]:
recs[ix].squeeze().shape

In [ ]:
recs[ix]

In [ ]:
# sns.lineplot(x=ts[0][:100], y=recs[ix][:100].squeeze().detach().cpu().numpy())
sns.lineplot(x=ts[0], y=recs[ix].squeeze().detach().cpu().numpy())

In [ ]:
truths.shape

In [ ]:
# sns.lineplot(x=ts[0][:100], y=truths[ix][:100].squeeze().detach().cpu().numpy())
sns.lineplot(x=ts[0], y=truths[ix].squeeze().detach().cpu().numpy())

### TSNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
z_us.shape

In [ ]:
latents = z_us.squeeze().detach().cpu().numpy()

In [ ]:
latents.shape

In [ ]:
tsne = TSNE(perplexity=10)

In [ ]:
xx = tsne.fit_transform(latents)

In [ ]:
sns.scatterplot(x=xx[:,0], y=xx[:,1])

In [ ]:
df = pd.DataFrame(xx, columns=['x1','x2'])
df['minimums'] = [np.round(x.min().item(),2) for x in truths]

In [ ]:
sns.scatterplot(x='x1', y='x2', data=df, hue='minimums')

In [ ]:
g=df.loc[(df['x1']<-100) & (df['x2']<-100)]
g=df.loc[(df['x1']>0) & (df['x2']>100)]

In [ ]:
gx = g.index

In [ ]:
len(truths)

In [ ]:
ls=truths[gx]

In [ ]:
for i, p in enumerate(truths):
    x = p.detach().cpu().squeeze().numpy()
    plt.figure(figsize=(5,3))
    sns.lineplot(x=ts[i], y=x)
    plt.show()